## Using manually classified timelapse camera data to find GOES multi-spectral thresholds

We want to find a combination of thresholds from the GOES Day Cloud Phase RGB bands that will help predict cloud/no cloud for all pixels over Western Washington. We have manually identified cloud types for daylight hours between 13 July 2022 and 30 September 2022 for Friday Harbor and Cattle Point.

#### Methods:
1. Create RGB composite files for all the desired dates
2. Match RGB composites for the pixel just East of cameras for corresponding timesteps
    - GOES data is 5min but cameras are 30min
    - one pixel East because cameras look East towards Lopez and Shaw islands ~1-2km away
3. Only look at times with blue sky only or 2 or fewer cloud types 
    - gets way too messy when there is patchy cloud cover/many types of clouds
4. Get rid of timesteps and combine both sites cloud data with their corresponding GOES pixels
5. Use a decision tree to find the thresholds

#### Next Steps:
1. Redownload GOES data for the same dates but a larger spatial domain
    - see manuscript for domain bounds
2. Create a cloud/no cloud frequency map
3. Compare to GOES cloud mask that only uses thermal
    - theoretically, this will be very different specifically over the ocean
    - GOES cloud mask confuses cold water with low clouds


In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score


In [2]:
# # For Cattle Point
# lat = 48.464462
# lon = -122.909918 # offset by ~0.05deg (1 grid cell) to east since that is where camera looks
# cp_camera_data = pd.read_csv('CP2022photos.csv', index_col=0, parse_dates=True)
# camera_data = cp_camera_data

# For Friday Harbor
lat = 48.546348
lon = -123.002626 # offset by ~0.05deg (1 grid cells) to east since that is where camera looks
fhl_camera_data = pd.read_csv('FHL2022photos.csv', index_col=0, parse_dates=True)
camera_data = fhl_camera_data

camera_data.index = camera_data.index.tz_localize('America/Los_Angeles')
camera_data.index = camera_data.index.round('5min')
camera_data = camera_data.loc['2022-07-13':'2022-09-30']



In [3]:
goes_data_july = {}
goes_df_july = {}
for i in range(13, 32):
    key = f"goes_data_july{i}"  # Dynamically generate the key
    filename = f"/storage/cdalden/goes/washington/goes17/rgb_composite/goes17_C02_C05_C13_rgb_washington_202207{i}.nc"  # Dynamically generate the filename
    goes_data_july[key] = xr.open_dataset(filename)

    # Select data and convert to dataframe
    df_key = f"goes_df_july{i}"
    goes_df_july[df_key] = (
        goes_data_july[key]
        .sel(latitude=lat, longitude=lon, method='nearest')
        .drop_vars(['latitude', 'longitude'])
        .to_dataframe()
    )
# Concatenate all dataframes into a single dataframe
goes_df_july_all = pd.concat(goes_df_july.values(), axis=0)

# Convert goes times to PDT
goes_df_july_all.index = goes_df_july_all.index.tz_localize('UTC').tz_convert('America/Los_Angeles')
goes_df_july_all.index = goes_df_july_all.index.round('5min')

# Subset by time range (0600 to 2000 hours)
goes_df_july_all = goes_df_july_all.between_time('06:00', '19:30')



In [4]:
goes_data_aug = {}
goes_df_aug = {}
for i in range(1, 32):
    # Make i two digits
    day = f"{i:02}"
    key = f"goes_data_aug{day}"
    filename = f"/storage/cdalden/goes/washington/goes17/rgb_composite/goes17_C02_C05_C13_rgb_washington_202208{day}.nc"  # Dynamically generate the filename
    goes_data_aug[key] = xr.open_dataset(filename)

    # Select data and convert to dataframe
    df_key = f"goes_df_aug{i}"
    goes_df_aug[df_key] = (
        goes_data_aug[key]
        .sel(latitude=lat, longitude=lon, method='nearest')
        .drop_vars(['latitude', 'longitude'])
        .to_dataframe()
    )
# Concatenate all dataframes into a single dataframe
goes_df_aug_all = pd.concat(goes_df_aug.values(), axis=0)

# Convert goes times to PDT
goes_df_aug_all.index = goes_df_aug_all.index.tz_localize('UTC').tz_convert('America/Los_Angeles')
goes_df_aug_all.index = goes_df_aug_all.index.round('5min')

# Subset by time range (0600 to 2000 hours)
goes_df_aug_all = goes_df_aug_all.between_time('06:00', '19:30')


In [5]:
goes_data_sep = {}
goes_df_sep = {}
for i in range(1, 31):
    # Make i two digits
    day = f"{i:02}"
    key = f"goes_data_sep{day}"  # Dynamically generate the key
    filename = f"/storage/cdalden/goes/washington/goes17/rgb_composite/goes17_C02_C05_C13_rgb_washington_202209{day}.nc"  # Dynamically generate the filename
    goes_data_sep[key] = xr.open_dataset(filename)

    # Select data and convert to dataframe
    df_key = f"goes_df_sep{day}"
    goes_df_sep[df_key] = (
        goes_data_sep[key]
        .sel(latitude=lat, longitude=lon, method='nearest')
        .drop_vars(['latitude', 'longitude'])
        .to_dataframe()
    )
# Concatenate all dataframes into a single dataframe
goes_df_sep_all = pd.concat(goes_df_sep.values(), axis=0)

# Convert goes times to PDT
goes_df_sep_all.index = goes_df_sep_all.index.tz_localize('UTC').tz_convert('America/Los_Angeles')
goes_df_sep_all.index = goes_df_sep_all.index.round('5min')

# Subset by time range (0600 to 2000 hours)
goes_df_sep_all = goes_df_sep_all.between_time('06:00', '19:30')

In [6]:
goes_df_sep_all

,green,blue,red
t,,,
2022-08-31 17:00:00-07:00,0.075175,0.013398,0.000000
2022-08-31 17:10:00-07:00,0.073976,0.006862,0.000000
2022-08-31 17:10:00-07:00,0.193896,0.272676,0.000000
2022-08-31 17:20:00-07:00,0.233869,0.368544,0.065687
2022-08-31 17:20:00-07:00,0.199093,0.274853,0.026842
...,...,...,...
2022-09-30 16:40:00-07:00,0.050384,0.000048,0.000000
2022-09-30 16:40:00-07:00,0.051168,0.002193,0.000000
2022-09-30 16:50:00-07:00,0.049201,0.000048,0.000000


In [3]:
# Combine July and August data
goes_df_summer_all = pd.concat([goes_df_july_all, goes_df_aug_all, goes_df_sep_all])
# Ensure the time indices align
data = goes_df_summer_all.join(camera_data, how='inner')
data = data.dropna()

# Drop timesteps with more than 2 cloud types
data = data[data['clouds'].str.len() <= 2]

# Drop times when bluesky and string is 2 characters long (i.e., bluesky + 1 cloud type)
# data = data[~((data['clouds'].str.len() == 2) & (data['clouds'].str.contains('b')))]

print(data['clouds'].value_counts(
))

NameError: name 'goes_df_july_all' is not defined

In [16]:
fhl_df = pd.read_csv('fhl_goes_camera_2022_summer.csv')
cp_df = pd.read_csv('cp_goes_camera_2022_summer.csv')

fhl_df.drop(columns='Unnamed: 0', inplace=True)
cp_df.drop(columns='Unnamed: 0', inplace=True)

data = pd.concat([fhl_df, cp_df], ignore_index=True)

# drop where clouds == 'sb' or 'rb'
data = data[~data['clouds'].isin(['sb', 'rb', 'bh', 'cb', 'fb', 'b!'])]

data

,green,blue,red,clouds
14,0.060023,0.000574,0.0,b
15,0.060833,0.007203,0.0,b
16,0.055563,0.009413,0.0,b
17,0.054347,0.000000,0.0,b
18,0.053942,0.000000,0.0,b
...,...,...,...,...
7162,0.128683,0.351769,0.0,b
7165,0.152290,0.413954,0.0,b
7166,0.145995,0.426818,0.0,b
7171,0.112152,0.287412,0.0,b


In [17]:
print(data['clouds'].value_counts(
))

clouds
b     2500
s     1528
f      266
sc     234
fs      78
sm      38
sh      28
m       28
d       16
h       14
sr      14
hm      10
r       10
cr       8
f!       4
c        2
Name: count, dtype: int64


In [18]:
# Group by time and clouds to count occurrences
cloud_counts = data.groupby([data.index, 'clouds']).size().unstack(fill_value=0)

# Normalize counts to get proportions
cloud_proportions = cloud_counts.div(cloud_counts.sum(axis=1), axis=0)

# # Plot the stacked area chart
# cloud_proportions.plot(kind='area', stacked=True, alpha=0.7, figsize=(10, 6))
# plt.xlabel('Time')
# plt.ylabel('Proportion')
# plt.title('Cloud Categories Proportion Over Time')
# plt.legend(title='Clouds')
# plt.show()

## Decision Tree

In [21]:
# Create binary target variable: 1 if 'clouds' contains 'b' (no clouds), 0 otherwise (clouds)
# data['no_clouds'] = (~data['clouds'].str.contains('b')).astype(int)
data['no_clouds'] = (~(data['clouds'] == 'b')).astype(int)

# Split into features (X) and target (y)
X = data[['red', 'green', 'blue']]  # Replace with your variable names
y = data['no_clouds']  # Binary target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a decision tree
tree_model = DecisionTreeClassifier(max_depth=2, min_samples_split=10, class_weight='balanced', random_state=42)
tree_model.fit(X_train, y_train)

# Visualize the decision tree rules
tree_rules = export_text(tree_model, feature_names=['red', 'green', 'blue'])
print("Decision Tree Rules:\n", tree_rules)

# Make predictions on the test set
y_pred = tree_model.predict(X_test)

Decision Tree Rules:
 |--- green <= 0.13
|   |--- red <= 0.00
|   |   |--- class: 0
|   |--- red >  0.00
|   |   |--- class: 1
|--- green >  0.13
|   |--- green <= 0.15
|   |   |--- class: 1
|   |--- green >  0.15
|   |   |--- class: 1



In [22]:
# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Evaluate metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.98      0.91       495
           1       0.97      0.82      0.89       461

    accuracy                           0.90       956
   macro avg       0.91      0.90      0.90       956
weighted avg       0.91      0.90      0.90       956

Accuracy: 0.9027196652719666
F1 Score: 0.8909730363423212
Precision: 0.9693877551020408
Recall: 0.824295010845987


In [23]:
# Print feature importance
feature_importances = tree_model.feature_importances_
for feature, importance in zip(['red', 'green', 'blue'], feature_importances):
    print(f"Feature: {feature}, Importance: {importance:.4f}")

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Print the training and evaluation sample sizes
print(f"Training sample size: {len(X_train)}")
print(f"Evaluation sample size: {len(X_test)}")

Feature: red, Importance: 0.3213
Feature: green, Importance: 0.6787
Feature: blue, Importance: 0.0000
Training sample size: 3344
Evaluation sample size: 1434


In [33]:
from sklearn.metrics import f1_score

# Define a function to classify based on the thresholds
def classify_based_on_thresholds(row):
    if row['green'] <= 0.15:
        if row['red'] <= 0.00:
            return 1  # Class 1 (no clouds)
    return 0  # Class 0 (clouds)

# Apply the function to the test set
manual_predictions = X_test.apply(classify_based_on_thresholds, axis=1)

# Calculate the F1 score
f1 = f1_score(y_test, manual_predictions)
print("F1 Score based on thresholds:", f1)

F1 Score based on thresholds: 0.10438799076212471


## Try to make 3 class threshold
Clear, low, cumulus

In [21]:
# Create multi-class target variable
conditions = [
    data['clouds'] == 'b',  # Class 0: clear sky
    data['clouds'].isin(['s', 'f', 'fs']),  # Class 1: stratus/fog - low clouds
    data['clouds'].isin(['cb', 'c', 'sc']),  # Class 2: cumulus/midish level clouds
]
choices = [0, 1, 2]

data['cloud_class'] = np.select(conditions, choices, default=-1)  # Default -1 for unexpected values
# Filter out rows with -1 in the target variable
data = data[data['cloud_class'] != -1]

# Split into features (X) and target (y)
X = data[['red', 'green', 'blue']]  # Replace with your variable names
y = data['cloud_class']  # Multi-class target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a decision tree
tree_model = DecisionTreeClassifier(max_depth=8, min_samples_split=10, class_weight='balanced', random_state=42)
tree_model.fit(X_train, y_train)

# Visualize the decision tree rules
tree_rules = export_text(tree_model, feature_names=['red', 'green', 'blue'])
print("Decision Tree Rules:\n", tree_rules)

# Make predictions on the test set
y_pred = tree_model.predict(X_test)

Decision Tree Rules:
 |--- green <= 0.13
|   |--- red <= 0.00
|   |   |--- green <= 0.06
|   |   |   |--- green <= 0.01
|   |   |   |   |--- green <= 0.00
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- green >  0.00
|   |   |   |   |   |--- green <= 0.01
|   |   |   |   |   |   |--- green <= 0.00
|   |   |   |   |   |   |   |--- green <= 0.00
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- green >  0.00
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- green >  0.00
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- green >  0.01
|   |   |   |   |   |   |--- class: 1
|   |   |   |--- green >  0.01
|   |   |   |   |--- blue <= 0.02
|   |   |   |   |   |--- green <= 0.03
|   |   |   |   |   |   |--- green <= 0.03
|   |   |   |   |   |   |   |--- blue <= 0.00
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- blue >  0.00
|   |   |   |   |   |   |   |   |--- class: 2
|   |   |   |  

In [22]:
# Detailed classification report for 3 classes
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Evaluate metrics for 3 classes
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score (weighted):", f1_score(y_test, y_pred, average='weighted'))
print("Precision (weighted):", precision_score(y_test, y_pred, average='weighted'))
print("Recall (weighted):", recall_score(y_test, y_pred, average='weighted'))

# Print feature importance
feature_importances = tree_model.feature_importances_
for feature, importance in zip(['red', 'green', 'blue'], feature_importances):
    print(f"Feature: {feature}, Importance: {importance:.4f}")

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Print the training and evaluation sample sizes
print(f"Training sample size: {len(X_train)}")
print(f"Evaluation sample size: {len(X_test)}")


Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.93       517
           1       0.89      0.73      0.80       365
           2       0.20      0.36      0.25        80

    accuracy                           0.80       962
   macro avg       0.67      0.67      0.66       962
weighted avg       0.85      0.80      0.82       962

Accuracy: 0.8035343035343036
F1 Score (weighted): 0.8231149629893824
Precision (weighted): 0.852712809114639
Recall (weighted): 0.8035343035343036
Feature: red, Importance: 0.2272
Feature: green, Importance: 0.5908
Feature: blue, Importance: 0.1820
Training sample size: 3364
Evaluation sample size: 1442
